#### Neighborhood Segmentation and Clustering: part 2

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import io

In [3]:
wiki_url= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(wiki_url)

#Generate data table, leads to dataframe creation: find table in scraped content
created_table = soup.find('table',{'class':'wikitable sortable'})
created_table;

In [5]:
rows_data = []
for row in created_table.find_all("tr"):
    col = row.find_all("td")
    col = [ele.text.strip() for ele in col]
    rows_data.append(col)

rows_data;

df_1 = pd.DataFrame(rows_data)
df_1 = df_1.rename(columns={0:"Postcode",1:"Borough",2:"Neighborhood"})
df_1.head();

#drop 'Bourough' = 'Not assigned'
df_2 = df_1.copy()
df_2 = df_1.drop([0])
df_2 = df_2.drop(df_2[df_2['Borough']=='Not assigned'].index)
df_2 = df_2.reset_index(drop=True)
df_2.head();

# Find Neighborhood' = 'Not assigned'
df_2.loc[6]

#Replace values where not assigned
df_3 = df_2.copy()

df_3['Neighborhood'] = df_3.apply(lambda row: row['Borough'] if row['Neighborhood'] == 'Not assigned' else row['Neighborhood'], axis=1 )

df_3.loc[6];

df_4 = df_3.copy()
df_4 = (df_4.groupby(['Postcode','Borough'])['Neighborhood'].apply(lambda x: ','.join(set(x.dropna()))).reset_index())
df_4 = pd.DataFrame(df_4)
df_4.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### Adding components for part spatial data and merging coordinates to the initial datafram 4 (df_4)

In [11]:
csv_data_url="http://cocl.us/Geospatial_data"
source=requests.get(csv_data_url).content
df_format=pd.read_csv(io.StringIO(source.decode('utf-8')))

In [14]:
df_format.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(df_format, df_4, on='Postcode')

df = df[['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
df.shape

(103, 5)